In [276]:
import pandas as pd
import numpy as np
import csv
import urllib
df=pd.read_excel('drive/My Drive/Elections 2020/ElectionResults.xlsx',sheet_name='Sheet1')
df=df.append(pd.DataFrame({'state':['alaska','washington dc'],'county':['alaska','washington dc'],'gopvotes':[179064,16306],'demvotes':[142569,285728],'gopvotesper':[53.6,5.3],'demvotesper':[42.6,93.2]}),ignore_index=True)
df2=pd.read_csv('https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/national/totals/nst-est2019-alldata.csv')
df2=df2[['NAME','POPESTIMATE2019']]
df2=df2.rename(columns={'NAME':'state','POPESTIMATE2019':'statepop'})
df['county']=list(map(lambda x:x.lower(),df['county'].tolist()))
df2['state']=list(map(lambda x:x.lower(),df2['state'].tolist()))
df=pd.merge(df,df2,how='left',left_on=['state'],right_on=['state'])
df['statepop']=df['statepop'].fillna(684498)
df['statepop']=list(map(int,df['statepop'].tolist()))
df2=pd.read_excel('drive/My Drive/Elections 2020/CountyPopulation.xlsx')
df2=df2.rename(columns={'STNAME':'state','CTYNAME':'county','POPESTIMATE2019':'countypop'})
df2['state']=list(map(lambda x:x.lower(),df2['state'].tolist()))
df2['county']=list(map(lambda x:x.lower(),df2['county'].tolist()))
df=pd.merge(df,df2,how='left',left_on=['state','county'],right_on=['state','county'])
df=df.fillna(684498)
df['countypop']=list(map(int,df['countypop'].tolist()))
df2=pd.read_excel('drive/My Drive/Elections 2020/CountyDetails.xlsx')
df2['state']=list(map(lambda x:x.lower(),df2['state'].tolist()))
df2['county']=list(map(lambda x:x.lower(),df2['county'].tolist()))
df=pd.merge(df,df2,how='left',left_on=['state','county'],right_on=['state','county'])
l1=np.asarray(df['countypop'].tolist(),dtype=int)
l2=np.asarray(df['poverty'].tolist(),dtype=int)
l=np.divide(l2,l1)
l=np.multiply(l,100)
l=list(map(lambda x:round(x,2),l))
df['povertyper']=l
l=list(set(df['state'].tolist()))
df_temp=pd.DataFrame(columns=['state','county','poverty','medianincome'])
for i in l:
    df_temp=df_temp.append(df2[(df2['state']==i) & (df2['county']==i)])
df_temp=df_temp.reset_index(drop=True)[['state','poverty','medianincome']].rename(columns={'poverty':'statepoverty','medianincome':'statemedianincome'})
df=pd.merge(df,df_temp,how='left',left_on='state',right_on='state')
l1=np.asarray(df['statepop'].tolist(),dtype=int)
l2=np.asarray(df['statepoverty'].tolist(),dtype=int)
l=np.divide(l2,l1)
l=np.multiply(l,100)
l=list(map(lambda x:round(x,2),l))
df['statepovertyper']=l
df=df.rename(columns={'poverty':'countypoverty','povertyper':'countypovertyper','medianincome':'countymedianincome'})
df['countytotalvotes']=np.add(np.asarray(df['demvotes'].tolist()),np.asarray(df['gopvotes'].tolist()))
df_stvt=df.groupby(['state']).sum()['countytotalvotes']
df_stvt=df_stvt.to_frame()
df_stvt.reset_index(inplace=True)
df_stvt=df_stvt.rename(columns={'countytotalvotes':'statetotalvotes'})
df=pd.merge(df,df_stvt,how='left',left_on='state',right_on='state')

In [277]:
df_fips=pd.read_csv('https://raw.githubusercontent.com/kjhealy/fips-codes/master/county_fips_master.csv',encoding='ISO-8859-1')
df_fips=df_fips[['fips','county_name','state_name']]
df_fips=df_fips.rename(columns={'county_name':'county','state_name':'state'})
df_fips['fips']=list(map(lambda x:('0'*(5-len(str(x))))+str(x),df_fips['fips'].tolist()))
df_fips['county']=list(map(lambda x:x.lower(),df_fips['county'].tolist()))
df_fips['state']=list(map(lambda x:x.lower(),df_fips['state'].tolist()))
df_fips=df_fips.replace(['doða ana county'],'doña ana county')
df_fips=df_fips.replace(['la salle parish'],'lasalle parish')
df_fips=df_fips.replace(['district of columbia'],'washington dc')

In [278]:
df=pd.merge(df,df_fips,how='left',left_on=['state','county'],right_on=['state','county'])
df['fips']=df['fips'].fillna('02000')

In [279]:
df=df.drop_duplicates(keep='first',ignore_index=True)

In [280]:
df_temp=df.groupby(['state']).sum()[['gopvotes','demvotes']]
df_temp.reset_index(inplace=True)
df_temp['statewinner']=list(map(lambda x:'Trump' if df_temp.loc[x]['gopvotes']>df_temp.loc[x]['demvotes'] else 'Biden',[i for i in range(len(df_temp))]))
df_temp=df_temp.rename(columns={'gopvotes':'stategopvotes','demvotes':'statedemvotes'})
df=pd.merge(df,df_temp,how='left',left_on='state',right_on='state')

In [281]:
df['countywinner']=list(map(lambda x:'Trump' if df.loc[x]['gopvotes']>df.loc[x]['demvotes'] else 'Biden',[i for i in range(len(df))]))
df=df.rename(columns={'gopvotes':'countygopvotes','demvotes':'countydemvotes','gopvotesper':'countygopvotesper','demvotesper':'countydemvotesper'})

In [ ]:
df_temp=pd.read_excel('/content/drive/My Drive/Elections 2020/ElectionStateResults.xlsx')
df_temp['stategopvotesper']=list(map(lambda x:x[:len(x)-1],df_temp['stategopvotesper'].tolist()))
df_temp['statedemvotesper']=list(map(lambda x:x[:len(x)-1],df_temp['statedemvotesper'].tolist()))
df=pd.merge(df,df_temp,how='left',left_on='state',right_on='state')

In [ ]:
templ=list(df.columns)
df=df[sorted(templ)]

In [ ]:
df['state']=list(map(lambda x:x.title(),df['state'].tolist()))
df['county']=list(map(lambda x:x.title(),df['county'].tolist()))

In [289]:
!pip install xlsxwriter
writer=pd.ExcelWriter('drive/My Drive/Elections 2020/Elections2020Data.xlsx',engine='xlsxwriter')
df.to_excel(writer,sheet_name='Sheet1',index=False)
writer.save()